In [31]:
import numpy as np
import random
from sys import exit
class MineField:
    def __init__(self):
        pass
    
    def new(self, size, mine_num, first=None): # size = w, h
        width, height = size
        self.field = np.empty((width, height),dtype=int)
        self.digged = []
        self.flags = []
        self.death = False
        self.quit = False
        
        nums = np.arange(width*height)
        
        if first is not None:
            nums = nums[~np.vectorize(lambda x: x in [w*height+h for w, h in self.adjacentcell(first)])(nums)]

        self.mines = [(point//height,point%height) for point in np.random.choice(nums, mine_num, replace=False)]

        for h in range(height):
            for w in range(width):
                self.field[w,h] = self.calcdigit((w,h))
        
    def draw(self, mine=False, mine_char='＊'):
        width, height = self.field.shape
        to_zenkaku = dict(zip(range(9),'〇１２３４５６７８'))
        
        print('　ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ'[:width+1])
        for h in range(height):
            #if h < 20: print('①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳'[h],end='')
            #else: print(h+1,end='')
            print(f'{h+1:2}',end='')
            for w in range(width):
                if mine and ((w,h) in self.mines):
                    char = mine_char
                elif (w,h) in self.digged:
                    char = to_zenkaku[self.field[w,h]]
                elif (w,h) in self.flags:
                    char = 'Ｐ'
                else:
                    char = '・'
                print(char,end='')
            print('')
    
    def start(self, size=(10,10), mine_num=10, renew=True, renew_first_safe=True):       
        #if not renew: ignore renew_first_safe
        if renew:
            from functools import reduce
            fieldsize = reduce(lambda x,y: x*y, size)
            if not 0 < mine_num <= fieldsize - 9:
                print('Too many or few mines')
                return
            if renew_first_safe:
                t=MineField(); t.new(size=size, mine_num=mine_num); t.draw(); del t
                self.quit=False
                first = self.command(size)
                if self.quit: return
                self.new(size=size, mine_num=mine_num, first=first)
                self.autodig(first)
            else:
                self.new(size=size, mine_num=mine_num)
            
        while len(self.digged) < self.field.size - len(self.mines) and not self.death and not self.quit:
            self.draw()
            self.dig()
        if self.quit: return
        self.draw(True)
        return 'You lose' if self.death else 'You win!'
    
    def command(self, size):
        width, height = size
        while True:
            stri = input('command: ')
            if 'quit' in stri.lower():
                self.quit = True
                return None, None
            try:
                w=ord(stri[0].upper())-ord('A')
                h=int(''.join([ch for ch in stri if ch.isdigit()]))-1
                if 0<=w < width and h < height: break
            except: pass
            try: exec(stri)
            except: pass
            #(continue)
        return (w,h)
    
    def dig(self):
        w,h=self.command(self.field.shape)
        if self.quit: return
        self.autodig((w,h))

    def autodig(self, point):
        if point in self.digged: return
        if point in self.mines: self.death = True
        self.digged.append(point)
        if self.field[point] == 0: [self.autodig(adjs) for adjs in self.adjacentcell(point)]
                      
    def calcdigit(self, point):
        return len(set(self.adjacentcell(point)) & set(self.mines))
            
    def adjacentcell(self, point):
        width, height = self.field.sh
        return [(x, y) for y in (h-1, h, h+1) for x in (w-1, w, w+1) if 0 <= x < width and 0 <= y < height]
    ape
        w, h = point
    def undo(self):
        self.digged = self.digged[:-1]
        self.death = False
        
    def toggleflag(self,cell):
        if cell in self.flags:
            self.flags.remove(cell)
        else:
            self.flags.append(cell)
    

IndentationError: unexpected indent (<ipython-input-31-f5eee43ce607>, line 108)

In [33]:
m = MineField()
m.start((10,10),90)

　ＡＢＣＤＥＦＧＨＩＪ
 1・・・・・・・・・・
 2・・・・・・・・・・
 3・・・・・・・・・・
 4・・・・・・・・・・
 5・・・・・・・・・・
 6・・・・・・・・・・
 7・・・・・・・・・・
 8・・・・・・・・・・
 9・・・・・・・・・・
10・・・・・・・・・・
command: j3
　ＡＢＣＤＥＦＧＨＩＪ
 1・・・・・・・・・・
 2・・・・・・・・５２
 3・・・・・・・・３〇
 4・・・・・・・・５２
 5・・・・・・・・・・
 6・・・・・・・・・・
 7・・・・・・・・・・
 8・・・・・・・・・・
 9・・・・・・・・・・
10・・・・・・・・・・
command: e7
　ＡＢＣＤＥＦＧＨＩＪ
 1＊＊＊＊＊＊＊＊＊＊
 2＊＊＊＊＊＊・＊５２
 3＊＊＊＊＊＊＊＊３〇
 4＊＊＊＊＊＊＊＊５２
 5＊＊＊＊＊＊＊＊＊＊
 6＊＊・＊＊＊＊＊＊＊
 7＊＊＊＊＊＊＊＊＊＊
 8＊＊＊＊＊＊＊＊・・
 9＊＊＊＊＊＊＊＊＊＊
10＊＊＊＊＊＊＊＊＊＊


'You lose'

In [29]:
print(m.mines)

[(2, 6), (2, 0), (9, 8), (3, 7), (5, 6), (6, 2), (9, 1), (7, 5), (9, 0), (9, 2), (6, 4), (1, 4), (5, 8), (8, 6), (8, 4)]


In [28]:
m.undo()

In [6]:
import tkinter
root = tkinter.Tk()
root.title("test")
root.geometry("400x300+1000+10")
root.mainloop()

In [7]:
def donothing():
    filewin = tkinter.Toplevel(root)
    button = tkinter.Button(filewin, text="Do nothing button")
    button.pack()

root = tkinter.Tk()
menubar = tkinter.Menu(root)
filemenu = tkinter.Menu(menubar, tearoff=0)
filemenu.add_command(label="New", command=donothing)
filemenu.add_command(label="Open", command=donothing)
filemenu.add_command(label="Save", command=donothing)
filemenu.add_command(label="Save as...", command=donothing)
filemenu.add_command(label="Close", command=donothing)

filemenu.add_separator()

filemenu.add_command(label="Exit", command=root.quit)
menubar.add_cascade(label="File", menu=filemenu)
editmenu = tkinter.Menu(menubar, tearoff=0)
editmenu.add_command(label="Undo", command=donothing)

editmenu.add_separator()

editmenu.add_command(label="Cut", command=donothing)
editmenu.add_command(label="Copy", command=donothing)
editmenu.add_command(label="Paste", command=donothing)
editmenu.add_command(label="Delete", command=donothing)
editmenu.add_command(label="Select All", command=donothing)

menubar.add_cascade(label="Edit", menu=editmenu)
helpmenu = tkinter.Menu(menubar, tearoff=0)
helpmenu.add_command(label="Help Index", command=donothing)
helpmenu.add_command(label="About...", command=donothing)
menubar.add_cascade(label="Help", menu=helpmenu)

root.config(menu=menubar)
root.mainloop()

In [119]:
import numpy as np
import random
from sys import exit
from tkinter import *

class MineField(Frame):
    def __init__(self,master=None):
        super().__init__(master)
        self.bomb_list = []
        self.new((9,9),10)
        self.display()
        
        
    def display(self):
        self.root_frame = Frame(self, relief = 'groove', borderwidth = 5, bg = 'LightGray')
        self.status_frame = Frame(self.root_frame, height = 50, relief = 'sunken' ,borderwidth = 3, bg = 'LightGray')
        self.status_frame_label = Label(self.status_frame,text='game start',bg = 'LightGray',font=("",24))
        self.status_frame_label.place(width = 280, height = 50)
        self.game_frame = Frame(self.root_frame, relief = 'sunken', borderwidth = 3, bg = 'LightGray')
        self.root_frame.pack()
        self.status_frame.pack(pady = 5, padx = 5, fill = 'x')
        self.game_frame.pack(pady = 5, padx = 5)
        
        i = 0
        self.frame_list = []
        self.num = 9
        for w in range(self.num):
            for h in range(self.num):
                frame = Frame(self.game_frame, width = 30, height = 30, bd = 3, relief = 'raised', bg = 'LightGray')
                frame.bind("<1>", self.left_click)
                frame.bind("<3>", self.right_click)   
                frame.bind("<2>", self.m_click)
                frame.num = i
                self.frame_list.append(frame)
                frame.grid(row=w, column=h)
                i += 1
        self.grid(column=0, row=0)

        
    def new(self, size, mine_num, first=None): # size = w, h
        width, height = size
        self.field = np.empty((width, height),dtype=int)
        self.digged = []
        self.flags = np.full((width, height),True)
        self.death = False
        self.quit = False
        
        nums = np.arange(width*height)
        
        if first is not None:
            nums = nums[~np.vectorize(lambda x: x in [w*height+h for w, h in self.adjacentcell(first)])(nums)]

        self.mines = [(point//height,point%height) for point in np.random.choice(nums, mine_num, replace=False)]

        for h in range(height):
            for w in range(width):
                self.field[w,h] = self.calcdigit((w,h))
        
#    def draw(self, mine=False, mine_char='＊'):
#        width, height = self.field.shape
#        to_zenkaku = dict(zip(range(9),'〇１２３４５６７８'))
        
#        print('　ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ'[:width+1])
#        for h in range(height):
            #if h < 20: print('①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳'[h],end='')
            #else: print(h+1,end='')
#            print(f'{h+1:2}',end='')
#            for w in range(width):
#                if mine and ((w,h) in self.mines):
#                    char = mine_char
#                elif (w,h) in self.digged:
#                    char = to_zenkaku[self.field[w,h]]
#                elif (w,h) in self.flags:
#                    char = 'Ｐ'
#                else:
#                    char = '・'
#                print(char,end='')
#            print('')
    
#    def start(self, size=(10,10), mine_num=10, renew=True, renew_first_safe=True):       
#        #if not renew: ignore renew_first_safe
#        if renew:
#            from functools import reduce
#            fieldsize = reduce(lambda x,y: x*y, size)
#            if not 0 < mine_num <= fieldsize - 9:
#               print('Too many or few mines')
#                return
#           if renew_first_safe:
#              t=MineField(); t.new(size=size, mine_num=mine_num); t.draw(); del t
#             self.quit=False
#                first = self.command(size)
#               if self.quit: return
#              self.new(size=size, mine_num=mine_num, first=first)
#                self.autodig(first)
#           else:
#              self.new(size=size, mine_num=mine_num)
            
#        while len(self.digged) < self.field.size - len(self.mines) and not self.death and not self.quit:
#            self.draw()
#            self.dig()
#        if self.quit: return
#        self.draw(True)
#        return 'You lose' if self.death else 'You win!'
    
#    def command(self, size):
#        width, height = size
#        while True:
#            stri = input('command: ')
#            if 'quit' in stri.lower():
#                self.quit = True
#                return None, None
#            try:
#                w=ord(stri[0].upper())-ord('A')
#                h=int(''.join([ch for ch in stri if ch.isdigit()]))-1
#                if 0<=w < width and h < height: break
#            except: pass
#            try: exec(stri)
#            except: pass
            #(continue)
#        return (w,h)
    
#    def dig(self):
#        w,h=self.command(self.field.shape)
#        if self.quit: return
#        self.autodig((w,h))

    def autodig(self, point):
        if point in self.digged: return
        if point in self.mines: self.death = True
        self.digged.append(point)
        if self.field[point] == 0: [self.autodig(adjs) for adjs in self.adjacentcell(point)]
                      
    def calcdigit(self, point):
        return len(set(self.adjacentcell(point)) & set(self.mines))
            
    def adjacentcell(self, point):
        width, height = self.field.shape
        w, h = point
        return [(x, y) for y in (h-1, h, h+1) for x in (w-1, w, w+1) if 0 <= x < width and 0 <= y < height]
#   def undo(self):
#       self.digged = self.digged[:-1]
#       self.death = False
        
#    def toggleflag(self,cell):
#        if cell in self.flags:
#            self.flags.remove(cell)
#        else:
#            self.flags.append(cell)
    
    def left_click(self,event):
        event.widget.configure(relief = 'ridge', bd = '1')
        cell = event.widget.num
        
        
        height = len(self.field)
        cell = (cell//height,cell%height)
        
        self.autodig(cell)
        
        self.flags[cell] = False
        self.drawcell(event)
        
        
        
    
    def drawcell(self, event):
#         w, h = cell
#         height = len(self.field)
#         cellnum = w*height+h
        cellnum = event.widget.num
        height = len(self.field)
        cell = (cellnum//height,cellnum%height)
        
        if cell in self.digged and cell in self.mines:
            self.frame_list[cellnum].configure(bg = 'red')
            self.status_frame_label = Label(self.status_frame,text='game over',bg = 'LightGray',font=("",24))
            self.status_frame_label.place(width = 280, height = 50)
            for i in self.frame_list:
                i.bind("<1>", self.stop)
                i.bind("<3>", self.stop)
        else:
            bomb_count_label = Label(event.widget, text = self.field[cell], bg = 'LightGray',font=("",20)) 
            bomb_count_label.place(width = 28, height = 28)
            event.widget.bind("<1>", self.stop)
            
            if np.sum(self.flags) == len(self.mines):
                self.status_frame_label = Label(self.status_frame,text='game clear',bg = 'LightGray',font=("",24))
                self.status_frame_label.place(width = 280, height = 50)
                for i in self.frame_list:
                    i.bind("<1>", self.stop)
                    i.bind("<3>", self.stop)
            
    def right_click(self,event):
        if event.widget["bg"] == "LightGray":
            event.widget["bg"] = "gold"
            event.widget.bind("<1>", self.stop)
        # 黄色になっていたら、元に戻す。
        else:
            event.widget["bg"] = "LightGray"
            event.widget.bind("<1>", self.left_click)
    
    def m_click(self,event):
        print(self.mines)

    def stop(self,event):
        pass

#         #地雷リスト作成
#         if len(self.bomb_list) == 0:
#             while len(self.bomb_list) != 10:
#                 bomb_num = random.randint(0,80)
#                 if bomb_num != except_num and (bomb_num in self.bomb_list) == False:
#                     self.bomb_list.append(bomb_num)
#         #地雷判定
#         bomb_count = self.search_bomb(event.widget.num)
#         if  bomb_count == 9 :
#             for i in self.bomb_list:
#                 self.frame_list[i].configure(bg = 'red')
#             #地雷後のボタン停止
#             for i in self.frame_list:
#                 i.bind("<1>", self.stop)
#                 i.bind("<3>", self.stop)
#             self.status_frame_label = Label(self.status_frame,text='game over',bg = 'LightGray',font=("",24))
#             self.status_frame_label.place(width = 280, height = 50)
#         else:
#             bomb_count_label = Label(event.widget, text = bomb_count, bg = 'LightGray',font=("",20)) 
#             bomb_count_label.place(width = 28, height = 28)
#             event.widget.bind("<1>", self.stop)
    

In [120]:
root = Tk()
root.title("マインスイーパ")
root.resizable(0,0)
MineField(root)
root.mainloop() 